# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [22]:
import os

In [23]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [24]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [25]:
# TODO: Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [32]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a clear and succint financial strategy with precise numbers and allocations.
"""

In [33]:
# Request from the client
request = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [34]:
Markdown(response.content)

Given that you are 25 years old and have $1,000 to invest, it's important to consider a strategy that balances potential growth with risk management. Here's a recommended financial strategy:

1. **Establish an Emergency Fund**: Before making investments, consider setting aside at least 3-6 months of expenses in a high-yield savings account. However, since you have $1,000, we will assume you are ready to invest.

2. **Diversification**: With a small initial investment, it's crucial to diversify to mitigate risks. Consider the following allocations within your $1,000:

   - **Index Funds / ETFs (60%)**: $600
     - **Example**: SPDR S&P 500 ETF (SPY) or Vanguard Total Stock Market ETF (VTI). These funds provide broad market exposure and tend to have lower fees.
   
   - **Growth Stocks (30%)**: $300
     - **Example**: Companies like Apple (AAPL), Amazon (AMZN), or Alphabet (GOOGL). These stocks have strong growth potential, but do come with higher risk.
   
   - **Dividend Stocks (10%)**: $100
     - **Example**: Companies like Johnson & Johnson (JNJ) or Procter & Gamble (PG) that provide consistent dividends. These could offer stability and modest returns.

3. **Investment Platform**: Use a platform with low or no trading fees (like Robinhood, M1 Finance, or Charles Schwab) to maximize your investment.

4. **Dollar-Cost Averaging**: To reduce the impact of volatility, consider investing a portion of your money each month rather than all at once.

5. **Continuous Learning and Reassessment**: Continue learning about investing and monitor your portfolio regularly to make adjustments based on market conditions and your financial goals.

### Summary Allocation:
- **Index Funds / ETFs**: $600
- **Growth Stocks**: $300
- **Dividend Stocks**: $100

This balanced approach allows you to start building your portfolio with a mix of risk and stability, suitable for your age and investment horizon. Always remember to conduct your own research or consult with a financial advisor before making investment decisions.

## Agentic system

We will now create a simple agentic system consisting of three agents using LangChain.
This system will help us perform a more elaborate financial analysis by including:

- Client Interface Agent: Rephrases the user’s prompt to improve the quality of the financial analyst’s response.

- Financial Analyst: Similar to the first part; provides financial advice based on the refined prompt.

- Risk Advisor: Assesses the risk associated with the advice given by the financial analyst.

In [35]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load LLM
llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

# Agent Prompts
client_prompt = PromptTemplate.from_template(
    "You are a Client Interface agent. A user has asked the following question:\n{user_query}\n"
    "Rephrase it clearly for a financial analyst."
)
analyst_prompt = PromptTemplate.from_template(
    "You are a Financial Analyst. The client asked: {client_rephrased_query}\n"
    "Give clear investment advice. Do not assess risk."
)
risk_prompt = PromptTemplate.from_template(
    "You are a Risk Advisor. The analyst has recommended:\n{analyst_advice}\n"
    "Evaluate this advice from a risk perspective. Offer any cautions and ways to reduce risk."
)

# Define Chains
client_chain = client_prompt | llm | StrOutputParser()
analyst_chain = analyst_prompt | llm | StrOutputParser()
risk_chain = risk_prompt | llm | StrOutputParser()

# Simulate user question
user_query = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Step 1: Client Interface processes input
client_rephrased_query = client_chain.invoke(user_query)
print("\n🤖 Client Interface:\n", client_rephrased_query)

# Step 2: Financial Analyst responds
analyst_advice = analyst_chain.invoke(client_rephrased_query)
print("\n📈 Financial Analyst:\n", analyst_advice)

# Step 3: Risk Advisor evaluates
risk_evaluation = risk_chain.invoke(analyst_advice)
print("\n⚠️ Risk Advisor:\n", risk_evaluation)


🤖 Client Interface:
 A 25-year-old individual has $1,000 in savings and is seeking recommendations for U.S. stocks to invest in.

📈 Financial Analyst:
 As a financial analyst providing investment recommendations, I’ll suggest a few U.S. stocks that have shown potential for growth. While it's important to conduct further research and consider diversifying investments, here are several options that a 25-year-old investor with $1,000 might consider:

1. **Technology Stocks**:
   - **Apple Inc. (AAPL)**: A well-established tech company with a strong product ecosystem that includes hardware, software, and services. Historically, Apple has shown consistent growth and innovation.
   - **Microsoft Corporation (MSFT)**: A leader in software, cloud computing, and gaming. Microsoft has a solid financial base and a strong presence in various markets.

2. **Consumer Discretionary Stocks**:
   - **Amazon.com, Inc. (AMZN)**: A dominant player in e-commerce and cloud computing (AWS). Amazon has a his

In [36]:
summarize_prompt = PromptTemplate.from_template(
    "You are a Client interface agent. Give a clear and succint financial strategy with precise numbers and allocations"
    "based on \n{analyst_advice}\n and \n{risk_evaluation}\n"
)

summarized_chain = {"client_rephrased_query": client_chain} | {"analyst_advice": analyst_chain} | {"risk_evaluation": risk_chain} | summarize_prompt | llm | StrOutputParser()

summary= summarized_chain.invoke(user_query)
print("\n⚖️ Summary:\n", summary)


⚖️ Summary:
 Here’s a clear and succinct financial strategy for your $1,000 investment, considering diversification, risk management, and a long-term growth perspective:

### Investment Strategy Overview

1. **Investment Amount**: $1,000

2. **Allocation Breakdown**:
   - **ETFs (Broad Market Exposure)**: 50% ($500)
     - **Vanguard Total Stock Market ETF (VTI)**: $250
     - **SPDR S&P 500 ETF Trust (SPY)**: $250
   - **Individual Stocks**: 30% ($300)
     - **Apple Inc. (AAPL)**: $100
     - **Microsoft Corporation (MSFT)**: $100
     - **Coca-Cola Company (KO)**: $100
   - **Sector-Specific Stocks**: 10% ($100)
     - **NextEra Energy, Inc. (NEE)**: $50
     - **Johnson & Johnson (JNJ)**: $50
   - **Cash Reserve / Emergency Fund**: 10% ($100)
     - Keep in a high-yield savings account for liquidity.

### Rationale Behind the Allocation:

1. **Diversification**: 
   - Allocating primarily to ETFs (50%) provides broad market exposure, reducing the risk associated with individual st